# Tensorflow 실습 : BERT 모델 이용하기

In [1]:
# bert 관련 import를 위해 설치
!pip install -q tf-models-nightly

     |████████████████████████████████| 870kB 5.9MB/s 
     |████████████████████████████████| 325.2MB 46kB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 174kB 49.7MB/s 
     |████████████████████████████████| 36.4MB 119kB/s 
     |████████████████████████████████| 358kB 41.2MB/s 
     |████████████████████████████████| 1.1MB 48.7MB/s 
     |████████████████████████████████| 6.7MB 49.5MB/s 
     |████████████████████████████████| 460kB 52.3MB/s 
     |████████████████████████████████| 296kB 49.1MB/s 


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import official.nlp.bert.tokenization

## BERT 모델 
- 기본적으로 [BERT](https://arxiv.org/pdf/1810.04805.pdf)(Bidirectional Encoder Representations from Transformers)는 Transformer Encoder의 네트워크 구조를 가지고 있음
  - input shape : batch, sequence length (정수 token sequence)
  - output shape : batch, sequence length, feature
  - pre-trained 모델을 다양한 task의 데이터셋에 fine-tuning 했을 때 좋은 성능을 보임
  - pre-training 방법
    - Masked Language Model (MLM): token의 일부를 가리고, 어떤 token이 들어갈지 맞추는 문제로 학습
    - Next Sentence Prediction (NSP): 두 개의 문장을 넣어주고, 두 문장이 이어지는 문장인지 아닌지 맞추는 문제로 학습
    


<table>
  <tr><td>
    <img src="https://mino-park7.github.io/images/2018/12/%EA%B7%B8%EB%A6%BC1-bert-openai-gpt-elmo-%EC%B6%9C%EC%B2%98-bert%EB%85%BC%EB%AC%B8.png" width="800">
  </td></tr>
  <tr><td align="center">
    <b>그림.</b> BERT의 네트워크 구조 (Encoder 구조라서 Bidirectional의 화살표를 모두 가지고 있음) <br/>&nbsp;
  </td></tr>
</table>


<table>
  <tr><td>
    <img src="https://raw.githubusercontent.com/jiabaogithub/imgs/master/img/bert20190926153602.png" width="800">
  </td></tr>
  <tr><td align="center">
    <b>그림.</b> BERT의 Pre-training과 Fine-Tuning <br/>&nbsp;
  </td></tr>
</table>


<table>
  <tr><td>
    <img src="https://mino-park7.github.io/images/2019/02/%EA%B7%B8%EB%A6%BC4-bert-experiment-result.png" width="800">
  </td></tr>
  <tr><td align="center">
    <b>그림.</b> BERT의 Fine-Tuning <br/>&nbsp;
  </td></tr>
</table>


### BERT (Multilingual)
- [BERT (Multilingual)](https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2)는 한국어, 영어 뿐 아니라 수많은 언어를 지원하는 BERT 모델
  - Multilingual Wikipedia dataset으로 학습이 된 모델 (pre-trained)
  - 한국어 전용 모델이 아니기 때문에 tokenizer의 성능이 좋지 않음
  - 한국어 전용 모델보다는 성능이 떨어지지만, 성능 자체는 나쁘지 않음
  - 해당 공개 모델에 대한 소개 [github](https://github.com/google-research/bert/blob/master/multilingual.md)
  - keras_hub의 multilingual 공개 모델: https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2

- 이외의 한국어 BERT 모델 (pre-trained)
  - [KoBERT](https://github.com/SKTBrain/KoBERT)
  - [KorBERT](http://aiopen.etri.re.kr/service_dataset.php)
  - [DistilKoBERT](https://github.com/monologg/DistilKoBERT)
  - [KoGPT-2](https://github.com/SKT-AI/KoGPT2) (BERT는 아니지만, 참고)


In [3]:
hub_url_bert = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2"
bert_layer = hub.KerasLayer(hub_url_bert, trainable=True)

- 최대 sequence 길이는 자유롭게 선택
  - 길이가 길면 GPU 메모리 error
- GPU 메모리 해결법
  - 최대 길이 줄이기
  - mini batch size 줄이기

In [4]:
# max_seq_length = 128  # Your choice here.
max_seq_length = 64
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")

- input으로 출력 결과 shape 확인
  - pooled_output: 모든 sequence에 대한 output state를 시간 축으로 pooling해서 출력
  - sequence_output: 각 time step에 대한 hidden state 출력 

In [5]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
print(pooled_output)
print(sequence_output)

Tensor("keras_layer/cond/Identity:0", shape=(None, 768), dtype=float32)
Tensor("keras_layer/cond/Identity_1:0", shape=(None, 64, 768), dtype=float32)


## 텍스트 전처리

### Tokenizer
- multilingual model이므로 vocab size가 굉장히 큼

In [6]:
# 저장된 tokenizer 불러오기
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = official.nlp.bert.tokenization.FullTokenizer(vocab_file, do_lower_case)

In [7]:
print("Vocab size:", len(tokenizer.vocab))

Vocab size: 119547


In [8]:
tokens = tokenizer.tokenize('english랑 한국어를 동시에 tokenize 해준다니!')
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

['engl', '##ish', '##랑', '한국', '##어를', '동시에', 'tok', '##eni', '##ze', '해', '##준', '##다', '##니', '!']
[20207, 15529, 62200, 48556, 80940, 58248, 18436, 18687, 10870, 9960, 54867, 11903, 25503, 106]


In [9]:
print(tokenizer.convert_ids_to_tokens(ids))

['engl', '##ish', '##랑', '한국', '##어를', '동시에', 'tok', '##eni', '##ze', '해', '##준', '##다', '##니', '!']


- `[CLS]`는 classification token 

=> **(중요) classification하기 위한 dense layer를 [CLS] token의 output hidden state에 계산하여 사용함**

- `[SEP]`는 문장 구분 token 
  - 기본적으로 BERT는 2개 문장이 input으로 들어온다고 가정함
- `[PAD]`는 의미 없는 뒷 부분 zero padding에 사용하는 token




In [10]:
tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]', '[PAD]'])

[101, 102, 0]

### BERT input 형식에 맞게 변환

<table>
  <tr><td>
    <img src="https://www.lyrn.ai/wp-content/uploads/2018/11/NSP.png" width="800">
  </td></tr>
  <tr><td align="center">
    <b>그림 .</b> BERT의 input <br/>&nbsp;
  </td></tr>
</table>


- 2개의 문장을 BERT의 input 형태로 적절하게 변환해주는 함수 (`bert_encode`)
- BERT는 input으로 총 3가지를 필요로 함
  - `input_word_ids`: 2개 문장을 token으로 표현한 token sequence (맨 앞에는 [CLS] token 추가, 각 문장의 끝에는 [SEP] token 추가) + 최대 sequence 길이에 맞추어 padding token
  - `input_mask`: padding이 아닌 곳은 1, padding인 곳은 0
  - `input_type_ids`: 문장을 구분해주는 표시, 첫 번째 문장은 0, 두번째 문장은 1

In [11]:
# 문장을 tokenize하고 마지막에 [SEP] token 추가
def encode_sentence(s, tokenizer):
  tokens = list(tokenizer.tokenize(s))
  tokens.append('[SEP]')
  return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(sentence1, sentence2, tokenizer, max_seq_length):
  sentence1 = encode_sentence(sentence1, tokenizer)
  sentence2 = encode_sentence(sentence2, tokenizer)

  # 추가할 paddding token 개수
  num_pad = max_seq_length - 1 - len(sentence1) - len(sentence2)

  # 전체 sequence의 길이는 max_seq_length 보다 작아야 함
  assert num_pad >= 0

  pad = tf.zeros(num_pad, dtype=tf.int32)

  # 제일 앞의 token은 [CLS], 다음은 첫번째 sentence, 두번째 sentence
  input_word_ids = tf.convert_to_tensor(tokenizer.convert_tokens_to_ids(['[CLS]']) + sentence1 + sentence2)
  # input mask는 padding이 아닌 부분은 1, padding은 0
  input_mask = tf.ones_like(input_word_ids)

  # type mask는 첫번째 문장은 0, 두번째 문장은 1
  type_cls = tf.zeros(1, dtype=tf.int32)
  type_s1 = tf.zeros_like(sentence1)
  type_s2 = tf.ones_like(sentence2)
  input_type_ids = tf.concat([type_cls, type_s1, type_s2], axis=0)

  # 모두 뒤에 zero padding 추가
  input_word_ids = tf.concat([input_word_ids, pad], axis=0)
  input_mask = tf.concat([input_mask, pad], axis=0)
  input_type_ids = tf.concat([input_type_ids, pad], axis=0)
  
  return input_word_ids, input_mask, input_type_ids

In [12]:
sentence1 = '안녕 bert 모델!'
sentence2 = '나는 너를 사랑해'

print(encode_sentence(sentence1, tokenizer))
print(encode_sentence(sentence2, tokenizer))
print(bert_encode(sentence1, sentence2, tokenizer, max_seq_length))

[9521, 118741, 10347, 10976, 9283, 118791, 106, 102]
[100585, 9004, 11513, 9405, 62200, 14523, 102]
(<tf.Tensor: shape=(64,), dtype=int32, numpy=
array([   101,   9521, 118741,  10347,  10976,   9283, 118791,    106,
          102, 100585,   9004,  11513,   9405,  62200,  14523,    102,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0],
      dtype=int32)>, <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
     

### 학습 데이터 예시

- 아래는 학습 데이터 예시
  - 두 개의 문장으로 각각의 데이터가 구성되고, 각 데이터에는 multi-class label이 달려있는 상황 가정

In [13]:
# 예시 input 데이터
sentences = [['나는 1번째 데이터의 1번째 문장이다.', '여기는 1번째 데이터의 2번째 문장이다.'],
             ['두번째의 1번째 문장이다.','2번째 데이터의 두번째 문장이겠지.'],
             ['그리고 여기 3번째 데이터의 첫번째 문장','3번째 데이터의 2번째 문장일까?!'],
             ['여기는 마지막 데이터의 첫 문자임', 'last data, second sentence']]

target_list = [1, 3, 4, 6]

sentences

[['나는 1번째 데이터의 1번째 문장이다.', '여기는 1번째 데이터의 2번째 문장이다.'],
 ['두번째의 1번째 문장이다.', '2번째 데이터의 두번째 문장이겠지.'],
 ['그리고 여기 3번째 데이터의 첫번째 문장', '3번째 데이터의 2번째 문장일까?!'],
 ['여기는 마지막 데이터의 첫 문자임', 'last data, second sentence']]

In [14]:
# bert_encode 함수를 이용하여, bert input으로 사용하기에 적절하게 전처리
ids_list, mask_list, type_list = list(), list(), list()


for s1, s2 in sentences:
  input_word_ids, input_mask, input_type_ids = bert_encode(s1, s2, tokenizer, max_seq_length)
  ids_list.append(input_word_ids)
  mask_list.append(input_mask)
  type_list.append(input_type_ids)

In [15]:
dataset = tf.data.Dataset.from_tensor_slices((ids_list, mask_list, type_list, target_list))

In [16]:
for input_word_ids, input_mask, input_type_ids, targets in dataset.take(1):
  print(input_word_ids)
  print(input_mask)
  print(input_type_ids)
  print(targets)

tf.Tensor(
[   101 100585    122  48506   9083  85297  10459    122  48506   9297
  55635  11903    119    102   9565  46216    122  48506   9083  85297
  10459    123  48506   9297  55635  11903    119    102      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0], shape=(64,), dtype=int32)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(64,), dtype=int32)
tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(64,), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)


In [17]:
for batch in dataset.batch(2).take(1):
  print(batch)

(<tf.Tensor: shape=(2, 64), dtype=int32, numpy=
array([[   101, 100585,    122,  48506,   9083,  85297,  10459,    122,
         48506,   9297,  55635,  11903,    119,    102,   9565,  46216,
           122,  48506,   9083,  85297,  10459,    123,  48506,   9297,
         55635,  11903,    119,    102,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0],
       [   101,   9102,  48506,  10459,    122,  48506,   9297,  55635,
         11903,    119,    102,    123,  48506,   9083,  85297,  10459,
          9102,  48506,   9297,  55635, 118632,  12508,    119,    102,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,

In [18]:
for input_word_ids, input_mask, segment_ids, targets in dataset.batch(2):
  pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
  print(pooled_output.shape)
  print(sequence_output.shape)

(2, 768)
(2, 64, 768)
(2, 768)
(2, 64, 768)


## pre-trained BERT를 fine tuning 하기

- fine-tuning을 위한 multi-class classifier로 만들기
  - [CLS] token은 sequence의 맨 앞에 있음
  - 맨 앞의 time step에 Dense layer를 이용하여 분류 logit 값을 계산

In [19]:
class BERT_classifier(tf.keras.Model):
    def __init__(self, num_class):
        super(BERT_classifier, self).__init__()
        self.bert_layer = hub.KerasLayer(hub_url_bert, trainable=True)
        self.dense = tf.keras.layers.Dense(num_class)

    def call(self, input_word_ids, input_mask, segment_ids, training=False):
        pooled_output, sequence_output = self.bert_layer([input_word_ids, input_mask, segment_ids], training=training)

        # [CLS] token이 sequence의 맨 앞에 있음
        cls_output = sequence_output[:, 0, :]
        logits = self.dense(cls_output)
        return logits

num_class = 10
model = BERT_classifier(num_class)

- multi-label classification인 경우, 아래와 같이 loss, optimizer 등을 선택하여 fine-tuning 

In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
training = True

for input_word_ids, input_mask, segment_ids, targets in dataset.batch(2):
    with tf.GradientTape() as tape:
        logits  = model(input_word_ids, input_mask, segment_ids, training=training)
        loss = loss_object(targets, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    print(logits.shape)

(2, 10)
(2, 10)


- 실제 fine-tuning 하는 예시는 [tensorflow 튜토리얼](https://www.tensorflow.org/official_models/fine_tuning_bert)을 확인